In [ ]:
!pip install spotipy
!pip install wheel
!pip install pandas
!pip install psycopg2

In [104]:
with open("C:/Users/Usuario/Documents/client_secret_spotify.txt",'r') as f:
    pwd= f.read()

In [105]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_id = '65a8d201a0fb46a095fa066b70f08123'
client_secret = pwd
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [119]:
# Creando la conexión a Redsshift
import psycopg2
url="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
data_base="data-engineer-database"
user="glo_tarcaya_coderhouse"
with open("C:/Users/Usuario/Documents/pwd_redshift.txt",'r') as f:
    pwd= f.read()
try:
    conn = psycopg2.connect(
        host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
        dbname=data_base,
        user=user,
        password=pwd,
        port='5439'
    )
    print("Conectado a Redshift con éxito!")
    
except Exception as e:
    print("No es posible conectar a Redshift")
    print(e)

Conectado a Redshift con éxito!


In [120]:
#Crear la tabla si no existe
with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS glo_tarcaya_coderhouse.canciones
        (
	    id VARCHAR(50) primary key  
	    ,artista VARCHAR(255)   
	    ,cancion VARCHAR(255)  
	    ,genero VARCHAR(300)   
	    ,album VARCHAR(100)   
	    ,total_canciones_album INTEGER  
	    ,Popularidad INTEGER 
	    ,fecha_lanzamiento date   
	    ,duracion_ms INTEGER   
	    ,album_img VARCHAR(300) 
        )
    """)
    conn.commit()

In [121]:
#Vaciar la tabla para evitar duplicados o inconsistencias
with conn.cursor() as cur:
  cur.execute("Truncate table canciones")
  count = cur.rowcount

In [122]:
#consultando la tabla
cur = conn.cursor()
cur.execute("SELECT * FROM canciones")
results = cur.fetchall()
results

[]

In [123]:
#Consultando los datos
import pandas as pd
results = sp.search(q='year:2023', type='track', limit=50)
data = {'Id': [],'Artista': [], 'Cancion': [],'Duracion_ms': [], 'Genero': [],'Album': [], 'Album_img': [], 'Total_canciones_album': [], 'Popularidad': [], 'fecha_lanzamiento': []}
for track in results['tracks']['items']:
    id = track['id']
    artist_name = track['artists'][0]['name']
    artist_id = track['artists'][0]['id']
    track_name = track['name']
    duration_ms = track['duration_ms']
    track_id = track['id']
    album_group = track['album']['name']
    album_img = track['album']['images'][0]['url'] #imagen de album
    album_cont = track['album']['total_tracks']
    track_genre = sp.artist(artist_id)['genres']
    track_popularity = track['popularity']
    track_year = track['album']['release_date']
    #Quitar las comillas 
    track_name = track_name.replace("'", "")
    album_group = album_group.replace("'", "")
    #Separar el género por coma
    track_genre = ', '.join(track_genre)

    data['Id'].append(id)
    data['Artista'].append(artist_name)
    data['Cancion'].append(track_name)
    data['Duracion_ms'].append(duration_ms)
    data['Album'].append(album_group)
    data['Album_img'].append(album_img)
    data['Total_canciones_album'].append(album_cont)
    data['Genero'].append(track_genre)
    data['Popularidad'].append(track_popularity)
    data['fecha_lanzamiento'].append(track_year)


df = pd.DataFrame(data)
#Evitar que haya canciones duplicadas
df.drop_duplicates(subset=['Artista', 'Cancion','Album'], keep='first', inplace=True)
#Reemplazar valores nulos o vacios en el campo Género por Desconocido
df['Genero'].fillna('Desconocido', inplace=True)
df.loc[df['Genero'] == '', 'Genero'] = 'Desconocido'
#Evitar que se cargue una canción con duración 0 ms
df = df[df['Duracion_ms'] != 0]
#Verificar que la fecha se muestre en formato fecha 
df['fecha_lanzamiento'] = pd.to_datetime(df['fecha_lanzamiento'], format='%Y-%m-%d')
display(df)

,Id,Artista,Cancion,Duracion_ms,Genero,Album,Album_img,Total_canciones_album,Popularidad,fecha_lanzamiento
0,6AQbmUe0Qwf5PZnt4HmTXv,PinkPantheress,Boys a liar Pt. 2,131013,Desconocido,Boys a liar Pt. 2,https://i.scdn.co/image/ab67616d0000b27342c5ba...,2,98,2023-02-03
1,3dnP0JxCgygwQH9Gm7q7nb,Eslabon Armado,Ella Baila Sola,165671,"corrido, corridos tumbados, nueva musica mexic...",Ella Baila Sola,https://i.scdn.co/image/ab67616d0000b2732071a0...,1,99,2023-03-16
2,59uQI0PADDKeE6UZDTJEe8,Morgan Wallen,Last Night,163854,contemporary country,3 Songs At A Time Sampler,https://i.scdn.co/image/ab67616d0000b273fc1df8...,3,91,2023-01-31
3,0yLdNVWF3Srea0uzk55zFn,Miley Cyrus,Flowers,200454,pop,Flowers,https://i.scdn.co/image/ab67616d0000b273f42954...,1,100,2023-01-13
4,7K3BhSpAxZBznislvUMVtn,Morgan Wallen,Last Night,163854,contemporary country,One Thing At A Time,https://i.scdn.co/image/ab67616d0000b273705079...,36,89,2023-03-03
5,7aRCf5cLOFN1U7kvtChY1G,Drake,Search & Rescue,272112,"canadian hip hop, canadian pop, hip hop, rap, ...",Search & Rescue,https://i.scdn.co/image/ab67616d0000b273cace8a...,1,91,2023-04-07
6,2UW7JaomAMuX9pZrjVpHAU,Yng Lvcas,La Bebe - Remix,234352,Desconocido,La Bebe (Remix),https://i.scdn.co/image/ab67616d0000b273a04be3...,2,97,2023-03-17
7,0PAcdVzhPO4gq1Iym9ESnK,Morgan Wallen,Thinkin’ Bout Me,177387,contemporary country,One Thing At A Time,https://i.scdn.co/image/ab67616d0000b273705079...,36,87,2023-03-03
8,2mpoNBkngUEwnaKoBhZ0vl,Peso Pluma,PRC,184066,"corrido, nueva musica mexicana, sad sierreno, ...",PRC,https://i.scdn.co/image/ab67616d0000b2737be314...,1,96,2023-01-23
9,0FA4wrjDJvJTTU8AepZTup,Lil Uzi Vert,Watch This - ARIZONATEARS Pluggnb Remix,163139,"melodic rap, philly rap, rap",Watch This (ARIZONATEARS Pluggnb Remix),https://i.scdn.co/image/ab67616d0000b2731e28b9...,1,90,2023-02-05


In [112]:
#Insertando los datos en Redsfhift
from psycopg2.extras import execute_values
with conn.cursor() as cur:
    execute_values(
        cur,
        '''
        INSERT INTO canciones (Id, Artista, Cancion, Duracion_ms, Genero, Album, Album_img, Total_canciones_album,Popularidad,fecha_lanzamiento)
        VALUES %s
        ''',
        [tuple(row) for row in df.values],
        page_size=len(df)
    )
    conn.commit()

In [114]:
cur.close()
conn.close()